# Paso a Paso: Implementando Pytest en un Proyecto de Data Engineering

En este notebook, vamos a seguir un proceso paso a paso para implementar pytest en un proyecto de Data Engineering. Veremos cómo configurar el entorno, escribir tests efectivos y ejecutarlos para verificar la calidad de nuestro código y datos.

Utilizaremos nuestro dataset de ventas de productos y crearemos un pequeño proyecto que incluya funciones para procesar y validar estos datos. Luego, implementaremos tests para asegurar que estas funciones funcionen correctamente.

## Paso 1: Configuración del Entorno

Lo primero que necesitamos es asegurarnos de tener instaladas todas las bibliotecas necesarias. Para este tutorial, necesitaremos pytest, pytest-cov (para medir la cobertura de código), pandas y numpy.

In [ ]:
# Instalamos las bibliotecas necesarias
!pip install pytest pytest-cov pandas numpy

## Paso 2: Estructura del Proyecto

Para este tutorial, ya hemos creado una estructura de proyecto típica para Data Engineering:

```
pytest_tutorial/
├── data/                  # Datos de ejemplo
│   └── ventas_productos.csv
├── notebooks/             # Jupyter notebooks para el tutorial
├── utils/                 # Módulos de utilidades y funciones
│   ├── __init__.py
│   ├── data_processing.py # Funciones para procesar datos
│   └── data_validation.py # Funciones para validar datos
└── tests/                 # Tests de pytest
    ├── __init__.py
    ├── test_processing.py # Tests para funciones de procesamiento
    └── test_validation.py # Tests para funciones de validación
```

Vamos a crear los directorios y archivos que faltan:

In [ ]:
# Creamos los directorios y archivos necesarios
!mkdir -p ../tests
!touch ../utils/__init__.py
!touch ../tests/__init__.py

## Paso 3: Exploración del Dataset

Antes de comenzar a escribir código y tests, vamos a explorar el dataset con el que trabajaremos. Esto nos ayudará a entender mejor los datos y a diseñar funciones y tests apropiados.

In [ ]:
import pandas as pd
import numpy as np

# Cargamos el dataset
df = pd.read_csv('../data/ventas_productos.csv')

# Mostramos las primeras filas
df.head()

In [ ]:
# Información básica del dataset
df.info()

In [ ]:
# Estadísticas descriptivas
df.describe()

## Paso 4: Creación de Funciones de Procesamiento de Datos

Ahora vamos a crear algunas funciones para procesar nuestros datos de ventas. Estas funciones estarán en el módulo `utils.data_processing`.

Primero, vamos a definir las funciones que queremos implementar:

1. `calcular_metricas_ventas`: Calcula métricas básicas de ventas (total, promedio, etc.)
2. `categorizar_productos_por_precio`: Categoriza productos según su precio
3. `calcular_tendencia_ventas`: Calcula la tendencia de ventas por mes

Vamos a implementar estas funciones:

In [ ]:
%%writefile ../utils/data_processing.py
import pandas as pd
import numpy as np

def calcular_metricas_ventas(df):
    """Calcula métricas de ventas a partir de un DataFrame de ventas.
    
    Args:
        df: DataFrame con columnas 'precio', 'cantidad', 'descuento' y 'total'
        
    Returns:
        dict: Diccionario con métricas calculadas
    """
    metricas = {
        'total_ventas': df['total'].sum(),
        'promedio_venta': df['total'].mean(),
        'total_productos_vendidos': df['cantidad'].sum(),
        'precio_promedio': df['precio'].mean(),
        'descuento_promedio': df['descuento'].mean(),
        'ahorro_total': (df['precio'] * df['cantidad'] * df['descuento']).sum()
    }
    return metricas

def categorizar_productos_por_precio(df):
    """Categoriza productos según su precio.
    
    Args:
        df: DataFrame con columna 'precio'
        
    Returns:
        DataFrame: DataFrame original con columna 'categoria_precio' añadida
    """
    df = df.copy()  # Evitamos modificar el DataFrame original
    
    # Definimos las categorías de precio
    condiciones = [
        (df['precio'] < 50),
        (df['precio'] >= 50) & (df['precio'] < 100),
        (df['precio'] >= 100) & (df['precio'] < 200),
        (df['precio'] >= 200)
    ]
    categorias = ['Económico', 'Estándar', 'Premium', 'Lujo']
    
    # Creamos la nueva columna
    df['categoria_precio'] = np.select(condiciones, categorias, default='Sin categoría')
    
    return df

def calcular_tendencia_ventas(df):
    """Calcula la tendencia de ventas por mes.
    
    Args:
        df: DataFrame con columnas 'fecha' y 'total'
        
    Returns:
        DataFrame: DataFrame con ventas totales por mes y variación porcentual
    """
    # Convertimos la columna fecha a datetime
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Extraemos el mes y agrupamos
    df['mes'] = df['fecha'].dt.to_period('M')
    ventas_mensuales = df.groupby('mes')['total'].sum().reset_index()
    ventas_mensuales['mes'] = ventas_mensuales['mes'].astype(str)
    
    # Calculamos la variación porcentual
    ventas_mensuales['variacion_porcentual'] = ventas_mensuales['total'].pct_change() * 100
    
    return ventas_mensuales

## Paso 5: Creación de Funciones de Validación de Datos

Ahora vamos a crear algunas funciones para validar nuestros datos. Estas funciones estarán en el módulo `utils.data_validation`.

Definiremos las siguientes funciones:

1. `validar_completitud`: Verifica que no haya valores nulos en las columnas requeridas
2. `validar_tipos_datos`: Verifica que las columnas tengan los tipos de datos esperados
3. `validar_rango_valores`: Verifica que los valores estén dentro de los rangos especificados

Vamos a implementar estas funciones:

In [ ]:
%%writefile ../utils/data_validation.py
import pandas as pd
import numpy as np

def validar_completitud(df, columnas_requeridas=None):
    """Valida que no haya valores nulos en las columnas requeridas.
    
    Args:
        df: DataFrame a validar
        columnas_requeridas: Lista de columnas a verificar. Si es None, se verifican todas.
        
    Returns:
        dict: Diccionario con resultados de la validación
    """
    if columnas_requeridas is None:
        columnas_requeridas = df.columns.tolist()
    
    # Verificamos que todas las columnas requeridas existan
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos valores nulos
    nulos_por_columna = {col: int(df[col].isnull().sum()) for col in columnas_requeridas}
    tiene_nulos = any(nulos_por_columna.values())
    
    if tiene_nulos:
        columnas_con_nulos = [col for col, nulos in nulos_por_columna.items() if nulos > 0]
        return {
            'valido': False,
            'error': f"Valores nulos encontrados en: {', '.join(columnas_con_nulos)}",
            'detalle': nulos_por_columna
        }
    
    return {
        'valido': True,
        'mensaje': "No se encontraron valores nulos en las columnas requeridas"
    }

def validar_tipos_datos(df, tipos_esperados):
    """Valida que las columnas tengan los tipos de datos esperados.
    
    Args:
        df: DataFrame a validar
        tipos_esperados: Diccionario con columnas y sus tipos esperados
        
    Returns:
        dict: Diccionario con resultados de la validación
    """
    # Verificamos que todas las columnas existan
    columnas_faltantes = [col for col in tipos_esperados.keys() if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos los tipos de datos
    tipos_incorrectos = {}
    for columna, tipo_esperado in tipos_esperados.items():
        tipo_actual = df[columna].dtype
        if not pd.api.types.is_dtype_equal(tipo_actual, tipo_esperado):
            # Para tipos numéricos, verificamos si podemos convertir sin pérdida de información
            if pd.api.types.is_numeric_dtype(tipo_esperado) and pd.api.types.is_numeric_dtype(tipo_actual):
                continue
            
            # Para fechas, verificamos si podemos convertir
            if tipo_esperado == 'datetime64[ns]' and pd.api.types.is_string_dtype(tipo_actual):
                try:
                    pd.to_datetime(df[columna])
                    continue
                except:
                    pass
            
            tipos_incorrectos[columna] = {'esperado': tipo_esperado, 'actual': str(tipo_actual)}
    
    if tipos_incorrectos:
        return {
            'valido': False,
            'error': f"Tipos de datos incorrectos en {len(tipos_incorrectos)} columnas",
            'detalle': tipos_incorrectos
        }
    
    return {
        'valido': True,
        'mensaje': "Todos los tipos de datos son correctos"
    }

def validar_rango_valores(df, rangos):
    """Valida que los valores estén dentro de los rangos especificados.
    
    Args:
        df: DataFrame a validar
        rangos: Diccionario con columnas y sus rangos (min, max)
        
    Returns:
        dict: Diccionario con resultados de la validación
    """
    # Verificamos que todas las columnas existan
    columnas_faltantes = [col for col in rangos.keys() if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos los rangos
    fuera_de_rango = {}
    for columna, (min_val, max_val) in rangos.items():
        # Verificamos mínimo
        if min_val is not None:
            valores_bajo_minimo = df[df[columna] < min_val]
            if not valores_bajo_minimo.empty:
                if columna not in fuera_de_rango:
                    fuera_de_rango[columna] = {}
                fuera_de_rango[columna]['bajo_minimo'] = {
                    'cantidad': len(valores_bajo_minimo),
                    'minimo_esperado': min_val,
                    'valor_minimo_encontrado': float(valores_bajo_minimo[columna].min())
                }
        
        # Verificamos máximo
        if max_val is not None:
            valores_sobre_maximo = df[df[columna] > max_val]
            if not valores_sobre_maximo.empty:
                if columna not in fuera_de_rango:
                    fuera_de_rango[columna] = {}
                fuera_de_rango[columna]['sobre_maximo'] = {
                    'cantidad': len(valores_sobre_maximo),
                    'maximo_esperado': max_val,
                    'valor_maximo_encontrado': float(valores_sobre_maximo[columna].max())
                }
    
    if fuera_de_rango:
        return {
            'valido': False,
            'error': f"Valores fuera de rango en {len(fuera_de_rango)} columnas",
            'detalle': fuera_de_rango
        }
    
    return {
        'valido': True,
        'mensaje': "Todos los valores están dentro de los rangos especificados"
    }

## Paso 6: Creación de Tests para Funciones de Procesamiento

Ahora que tenemos nuestras funciones de procesamiento, vamos a crear tests para verificar que funcionan correctamente. Estos tests estarán en el archivo `tests/test_processing.py`.

In [ ]:
%%writefile ../tests/test_processing.py
import pytest
import pandas as pd
import numpy as np
import sys
import os

# Añadimos el directorio raíz al path para poder importar los módulos
sys.path.append(os.path.abspath('..'))

from utils.data_processing import calcular_metricas_ventas, categorizar_productos_por_precio, calcular_tendencia_ventas

@pytest.fixture
def df_ventas_sample():
    """Fixture que crea un pequeño DataFrame de muestra para testing."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_ventas_completo():
    """Fixture que carga el dataset completo de ventas."""
    return pd.read_csv(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'ventas_productos.csv'))

def test_calcular_metricas_ventas(df_ventas_sample):
    """Test para la función de cálculo de métricas de ventas."""
    metricas = calcular_metricas_ventas(df_ventas_sample)
    
    # Verificamos que todas las métricas esperadas estén presentes
    assert set(metricas.keys()) == {'total_ventas', 'promedio_venta', 'total_productos_vendidos', 
                                    'precio_promedio', 'descuento_promedio', 'ahorro_total'}
    
    # Verificamos algunos cálculos específicos
    assert metricas['total_ventas'] == pytest.approx(1516.94, 0.01)
    assert metricas['total_productos_vendidos'] == 6
    assert metricas['precio_promedio'] == pytest.approx(403.32, 0.01)

def test_calcular_metricas_ventas_dataset_completo(df_ventas_completo):
    """Test para la función de cálculo de métricas con el dataset completo."""
    metricas = calcular_metricas_ventas(df_ventas_completo)
    
    # Verificamos que todas las métricas sean números válidos
    for metrica, valor in metricas.items():
        assert isinstance(valor, (int, float))
        assert not np.isnan(valor)
        assert not np.isinf(valor)

def test_categorizar_productos_por_precio(df_ventas_sample):
    """Test para la función de categorización de productos por precio."""
    df_categorizado = categorizar_productos_por_precio(df_ventas_sample)
    
    # Verificamos que se haya añadido la columna de categoría de precio
    assert 'categoria_precio' in df_categorizado.columns
    
    # Verificamos que las categorías sean correctas
    assert df_categorizado.loc[0, 'categoria_precio'] == 'Lujo'  # Laptop HP: 899.99
    assert df_categorizado.loc[1, 'categoria_precio'] == 'Premium'  # Monitor Dell: 249.99
    assert df_categorizado.loc[2, 'categoria_precio'] == 'Estándar'  # Teclado Logitech: 59.99
    
    # Verificamos que el DataFrame original no se haya modificado
    assert 'categoria_precio' not in df_ventas_sample.columns

def test_categorizar_productos_dataset_completo(df_ventas_completo):
    """Test para la función de categorización con el dataset completo."""
    df_categorizado = categorizar_productos_por_precio(df_ventas_completo)
    
    # Verificamos que se haya añadido la columna de categoría de precio
    assert 'categoria_precio' in df_categorizado.columns
    
    # Verificamos que todas las filas tengan una categoría válida
    categorias_validas = {'Económico', 'Estándar', 'Premium', 'Lujo', 'Sin categoría'}
    assert set(df_categorizado['categoria_precio'].unique()).issubset(categorias_validas)
    
    # Verificamos que haya al menos un producto en cada categoría principal
    for categoria in ['Económico', 'Estándar', 'Premium', 'Lujo']:
        assert categoria in df_categorizado['categoria_precio'].values, f"No hay productos en la categoría {categoria}"

def test_calcular_tendencia_ventas(df_ventas_sample):
    """Test para la función de cálculo de tendencia de ventas."""
    tendencia = calcular_tendencia_ventas(df_ventas_sample)
    
    # Verificamos que el DataFrame tenga las columnas esperadas
    assert set(tendencia.columns) == {'mes', 'total', 'variacion_porcentual'}
    
    # Verificamos que solo haya un mes (ya que todas las fechas son de enero 2023)
    assert len(tendencia) == 1
    assert tendencia.iloc[0]['mes'] == '2023-01'
    assert tendencia.iloc[0]['total'] == pytest.approx(1516.94, 0.01)
    assert np.isnan(tendencia.iloc[0]['variacion_porcentual'])  # No hay mes anterior para comparar

def test_calcular_tendencia_dataset_completo(df_ventas_completo):
    """Test para la función de tendencia con el dataset completo."""
    tendencia = calcular_tendencia_ventas(df_ventas_completo)
    
    # Verificamos que el DataFrame tenga las columnas esperadas
    assert set(tendencia.columns) == {'mes', 'total', 'variacion_porcentual'}
    
    # Verificamos que haya al menos un mes
    assert len(tendencia) > 0
    
    # Verificamos que los meses estén en formato YYYY-MM
    import re
    for mes in tendencia['mes']:
        assert re.match(r'^\d{4}-\d{2}$', mes), f"El mes {mes} no tiene el formato esperado (YYYY-MM)"

## Paso 7: Creación de Tests para Funciones de Validación

Ahora vamos a crear tests para nuestras funciones de validación de datos. Estos tests estarán en el archivo `tests/test_validation.py`.

In [ ]:
%%writefile ../tests/test_validation.py
import pytest
import pandas as pd
import numpy as np
import sys
import os

# Añadimos el directorio raíz al path para poder importar los módulos
sys.path.append(os.path.abspath('..'))

from utils.data_validation import validar_completitud, validar_tipos_datos, validar_rango_valores

@pytest.fixture
def df_valido():
    """Fixture que crea un DataFrame válido para testing."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_con_nulos():
    """Fixture que crea un DataFrame con valores nulos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', None, '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', None],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, None],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_tipos_incorrectos():
    """Fixture que crea un DataFrame con tipos de datos incorrectos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': ['899.99', '249.99', '59.99'],  # Strings en lugar de float
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_fuera_de_rango():
    """Fixture que crea un DataFrame con valores fuera de rango."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, -3],  # Valor negativo
        'descuento': [0.05, 0.00, 1.5],  # Descuento mayor a 1 (100%)
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_ventas_completo():
    """Fixture que carga el dataset completo de ventas."""
    return pd.read_csv(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'ventas_productos.csv'))

def test_validar_completitud_sin_nulos(df_valido):
    """Test para validar completitud en un DataFrame sin valores nulos."""
    resultado = validar_completitud(df_valido)
    assert resultado['valido'] is True
    assert 'mensaje' in resultado

def test_validar_completitud_con_nulos(df_con_nulos):
    """Test para validar completitud en un DataFrame con valores nulos."""
    resultado = validar_completitud(df_con_nulos)
    assert resultado['valido'] is False
    assert 'error' in resultado
    assert 'detalle' in resultado
    assert resultado['detalle']['fecha'] == 1
    assert resultado['detalle']['categoria'] == 1
    assert resultado['detalle']['cantidad'] == 1

def test_validar_completitud_columnas_especificas(df_con_nulos):
    """Test para validar completitud solo en columnas específicas."""
    # Validamos solo columnas que no tienen nulos
    resultado = validar_completitud(df_con_nulos, ['id', 'producto', 'precio', 'descuento', 'total'])
    assert resultado['valido'] is True
    
    # Validamos una columna que tiene nulos
    resultado = validar_completitud(df_con_nulos, ['id', 'fecha'])
    assert resultado['valido'] is False
    assert resultado['detalle']['fecha'] == 1

def test_validar_completitud_dataset_completo(df_ventas_completo):
    """Test para validar completitud en el dataset completo."""
    resultado = validar_completitud(df_ventas_completo)
    assert resultado['valido'] is True, "El dataset completo no debería tener valores nulos"

def test_validar_tipos_datos_correctos(df_valido):
    """Test para validar tipos de datos correctos."""
    tipos_esperados = {
        'id': 'int64',
        'precio': 'float64',
        'cantidad': 'int64',
        'descuento': 'float64',
        'total': 'float64'
    }
    resultado = validar_tipos_datos(df_valido, tipos_esperados)
    assert resultado['valido'] is True

def test_validar_tipos_datos_incorrectos(df_tipos_incorrectos):
    """Test para validar tipos de datos incorrectos."""
    tipos_esperados = {
        'id': 'int64',
        'precio': 'float64',  # En df_tipos_incorrectos, precio es string
        'cantidad': 'int64',
        'descuento': 'float64',
        'total': 'float64'
    }
    resultado = validar_tipos_datos(df_tipos_incorrectos, tipos_esperados)
    assert resultado['valido'] is False
    assert 'precio' in resultado['detalle']

def test_validar_tipos_datos_dataset_completo(df_ventas_completo):
    """Test para validar tipos de datos en el dataset completo."""
    tipos_esperados = {
        'id': 'int64',
        'fecha': 'object',  # Las fechas se cargan como strings (object)
        'producto': 'object',
        'categoria': 'object',
        'precio': 'float64',
        'cantidad': 'int64',
        'descuento': 'float64',
        'total': 'float64'
    }
    resultado = validar_tipos_datos(df_ventas_completo, tipos_esperados)
    assert resultado['valido'] is True, "Los tipos de datos en el dataset completo deberían ser correctos"

def test_validar_rango_valores_dentro_de_rango(df_valido):
    """Test para validar rangos de valores dentro de los límites."""
    rangos = {
        'precio': (0, 1000),
        'cantidad': (0, 10),
        'descuento': (0, 1),
        'total': (0, 1000)
    }
    resultado = validar_rango_valores(df_valido, rangos)
    assert resultado['valido'] is True

def test_validar_rango_valores_fuera_de_rango(df_fuera_de_rango):
    """Test para validar rangos de valores fuera de los límites."""
    rangos = {
        'precio': (0, 1000),
        'cantidad': (0, 10),  # En df_fuera_de_rango, hay un valor negativo
        'descuento': (0, 1),  # En df_fuera_de_rango, hay un descuento > 1
        'total': (0, 1000)
    }
    resultado = validar_rango_valores(df_fuera_de_rango, rangos)
    assert resultado['valido'] is False
    assert 'cantidad' in resultado['detalle']
    assert 'descuento' in resultado['detalle']
    assert 'bajo_minimo' in resultado['detalle']['cantidad']
    assert 'sobre_maximo' in resultado['detalle']['descuento']

def test_validar_rango_valores_dataset_completo(df_ventas_completo):
    """Test para validar rangos de valores en el dataset completo."""
    rangos = {
        'precio': (0, 1000),
        'cantidad': (0, 10),
        'descuento': (0, 1),
        'total': (0, 1000)
    }
    resultado = validar_rango_valores(df_ventas_completo, rangos)
    assert resultado['valido'] is True, "Todos los valores en el dataset completo deberían estar dentro de los rangos especificados"

## Paso 8: Ejecución de Tests

Ahora que tenemos nuestras funciones y tests, vamos a ejecutar los tests para verificar que todo funciona correctamente.

Primero, vamos a ejecutar los tests de procesamiento:

In [ ]:
!cd .. && python -m pytest tests/test_processing.py -v

Ahora, vamos a ejecutar los tests de validación:

In [ ]:
!cd .. && python -m pytest tests/test_validation.py -v

También podemos ejecutar todos los tests a la vez:

In [ ]:
!cd .. && python -m pytest

## Paso 9: Medición de Cobertura de Código

Una métrica importante en testing es la cobertura de código, que mide qué porcentaje de nuestro código está siendo ejecutado por los tests. Vamos a medir la cobertura de nuestros tests:

In [ ]:
!cd .. && python -m pytest --cov=utils

Para obtener un informe más detallado de la cobertura:

In [ ]:
!cd .. && python -m pytest --cov=utils --cov-report=term-missing

## Paso 10: Uso de Marcadores para Categorizar Tests

Los marcadores nos permiten categorizar los tests y seleccionar subconjuntos específicos para ejecutar. Vamos a crear un nuevo archivo de tests con marcadores:

In [ ]:
%%writefile ../tests/test_with_markers.py
import pytest
import pandas as pd
import numpy as np
import time
import sys
import os

# Añadimos el directorio raíz al path para poder importar los módulos
sys.path.append(os.path.abspath('..'))

@pytest.fixture
def df_ventas():
    """Fixture que carga el dataset de ventas."""
    return pd.read_csv(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'ventas_productos.csv'))

@pytest.mark.rapido
def test_columnas_requeridas(df_ventas):
    """Test rápido que verifica que el DataFrame tenga las columnas requeridas."""
    columnas_requeridas = ['id', 'fecha', 'producto', 'categoria', 'precio', 'cantidad', 'descuento', 'total']
    for columna in columnas_requeridas:
        assert columna in df_ventas.columns, f"La columna {columna} debería estar presente"

@pytest.mark.lento
def test_calculo_intensivo(df_ventas):
    """Test lento que simula un cálculo intensivo."""
    # Simulamos un cálculo que toma tiempo
    time.sleep(2)
    
    # Realizamos algún cálculo con el DataFrame
    resultado = df_ventas.groupby(['categoria', 'fecha']).agg({'total': 'sum'}).reset_index()
    assert not resultado.empty, "El resultado no debería estar vacío"

@pytest.mark.validacion
def test_valores_no_negativos(df_ventas):
    """Test que verifica que no haya valores negativos en columnas numéricas."""
    columnas_numericas = ['precio', 'cantidad', 'total']
    for columna in columnas_numericas:
        assert (df_ventas[columna] >= 0).all(), f"No debería haber valores negativos en {columna}"

Para ejecutar solo los tests rápidos:

In [ ]:
!cd .. && python -m pytest tests/test_with_markers.py -m rapido -v

Para ejecutar solo los tests de validación:

In [ ]:
!cd .. && python -m pytest tests/test_with_markers.py -m validacion -v

## Paso 11: Configuración de Pytest

Podemos configurar pytest para personalizar su comportamiento. Esto se hace típicamente a través de un archivo `pytest.ini` o `conftest.py`. Vamos a crear un archivo `pytest.ini` básico:

In [ ]:
%%writefile ../pytest.ini
[pytest]
markers =
    rapido: tests que se ejecutan rápidamente
    lento: tests que toman más tiempo en ejecutarse
    validacion: tests que validan la calidad de los datos

testpaths = tests
python_files = test_*.py
python_classes = Test*
python_functions = test_*

# Opciones de verbosidad y formato de salida
addopts = -v --no-header

## Paso 12: Creación de un Script para Ejecutar Tests

Para facilitar la ejecución de tests, podemos crear un script que ejecute los tests y genere un informe de cobertura:

In [ ]:
%%writefile ../run_tests.py
#!/usr/bin/env python
import subprocess
import sys
import os

def run_tests():
    """Ejecuta los tests y genera un informe de cobertura."""
    print("Ejecutando tests...")
    result = subprocess.run(["pytest", "--cov=utils", "--cov-report=term-missing"], capture_output=True, text=True)
    
    print("\nResultados de los tests:")
    print(result.stdout)
    
    if result.stderr:
        print("\nErrores:")
        print(result.stderr)
    
    return result.returncode

if __name__ == "__main__":
    # Cambiamos al directorio raíz del proyecto
    os.chdir(os.path.dirname(os.path.abspath(__file__)))
    
    # Ejecutamos los tests
    exit_code = run_tests()
    
    # Salimos con el código de retorno de pytest
    sys.exit(exit_code)

Hacemos el script ejecutable:

In [ ]:
!chmod +x ../run_tests.py

Y lo ejecutamos:

In [ ]:
!cd .. && python run_tests.py

## Conclusión

En este notebook, hemos seguido un proceso paso a paso para implementar pytest en un proyecto de Data Engineering. Hemos visto cómo:

1. Configurar el entorno y la estructura del proyecto
2. Explorar el dataset para entender los datos
3. Crear funciones de procesamiento y validación de datos
4. Escribir tests para verificar estas funciones
5. Ejecutar los tests y medir la cobertura de código
6. Usar marcadores para categorizar y seleccionar tests
7. Configurar pytest para personalizar su comportamiento
8. Crear un script para facilitar la ejecución de tests

Este enfoque sistemático nos permite asegurar la calidad de nuestro código y datos, lo que es fundamental en proyectos de Data Engineering donde la precisión y confiabilidad son críticas.

En el siguiente notebook, veremos ejemplos más avanzados de pytest aplicados a escenarios específicos de Data Engineering.